In [1]:
import pandas as pd
import numpy as np
import time
from IPython.display import display
import matplotlib.pyplot as plt
from pandasql import sqldf
from sklearn import linear_model
import statsmodels.api as sm
from scipy import stats
import seaborn
from scipy.stats import pearsonr
from datetime import datetime, timedelta
import math

***1. Test Assets***

In [2]:
# We use S&P1500
data = pd.read_csv("merged_final.csv")
data

,permno,yyyymm,monthid,ticker,gvkey,cusip,naics,gsubind,IM,range_20,...,ret_f3,ret_f4,ret_f5,ret_f6,ret_f7,ret_f8,ret_f9,ret_f10,ret_f11,ret_f12
0,10026,198603.0,75.0,JJSF,12825,466032109,311812,30202030,0.201498,0.146460,...,-0.156250,-0.375000,-0.066667,-0.166667,0.114286,0.051282,-0.048780,0.615385,0.031746,0.030769
1,10026,198604.0,76.0,JJSF,12825,466032109,311812,30202030,0.985464,0.158181,...,-0.375000,-0.066667,-0.166667,0.114286,0.051282,-0.048780,0.615385,0.031746,0.030769,-0.119403
2,10026,198605.0,77.0,JJSF,12825,466032109,311812,30202030,0.715998,0.158346,...,-0.066667,-0.166667,0.114286,0.051282,-0.048780,0.615385,0.031746,0.030769,-0.119403,-0.042373
3,10026,198606.0,78.0,JJSF,12825,466032109,311812,30202030,0.672322,0.126541,...,-0.166667,0.114286,0.051282,-0.048780,0.615385,0.031746,0.030769,-0.119403,-0.042373,0.159292
4,10026,198607.0,79.0,JJSF,12825,466032109,311812,30202030,0.638727,0.127120,...,0.114286,0.051282,-0.048780,0.615385,0.031746,0.030769,-0.119403,-0.042373,0.159292,0.114504
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
401583,93429,201802.0,458.0,CBOE,184500,12503M108,523210,40203040,0.516443,0.089356,...,-0.083817,0.066728,-0.066686,0.040976,-0.048016,0.176011,-0.043598,-0.090968,-0.046612,0.031629
401584,93429,201803.0,459.0,CBOE,184500,12503M108,523210,40203040,0.562365,0.340422,...,0.066728,-0.066686,0.040976,-0.048016,0.176011,-0.043598,-0.090968,-0.046612,0.031629,-0.004900
401585,93429,201804.0,460.0,CBOE,184500,12503M108,523210,40203040,0.544807,0.193830,...,-0.066686,0.040976,-0.048016,0.176011,-0.043598,-0.090968,-0.046612,0.031629,-0.004900,0.064648
401586,93429,201805.0,461.0,CBOE,184500,12503M108,523210,40203040,0.455098,0.122062,...,0.040976,-0.048016,0.176011,-0.043598,-0.090968,-0.046612,0.031629,-0.004900,0.064648,0.071253


**2. Data Screening**

In [3]:
# function is called later
def condition_first_element(group):
    # Check the first element of each group
    return group.iloc[0]['PRC'] > 5 and group.iloc[0]['MKTCAP'] > 100000000

#df = data.groupby(['monthid', 'ticker']).filter(condition_first_element)
#df

**3. 10 - 20 Factors**  
Factors include: A.1.1 - A.1.5, A.1.7 (revenue surprises / Rs), A.1.8 (tax surprises / tes), A.1.10 (# of quarters with consecutive earnings increase / nei), A.2.18 (5-year sales growth rank / Sr), A.2.19 (sales growth / Sg), Sentiment Analysis Factor

In [4]:
a15 = pd.read_csv("factorsA1_to_A5.csv")
a15

,Unnamed: 0,permno,yyyymm,monthid,ticker,conm,gvkey,cusip,naics,gsubind,...,ret_f8,ret_f9,ret_f10,ret_f11,ret_f12,Sue,abr,Res,R6,R11
0,0,76868.0,199102.0,134.0,AAON,b'AAON INC',b'021542',000360206,b'333415',b'20102010',...,-0.052632,0.166667,-0.095238,0.263158,-0.166667,NaN,4.0,NaN,NaN,NaN
1,1,76868.0,199103.0,135.0,AAON,b'AAON INC',b'021542',000360206,b'333415',b'20102010',...,0.166667,-0.095238,0.263158,-0.166667,0.050000,NaN,4.0,NaN,NaN,NaN
2,2,76868.0,199104.0,136.0,AAON,b'AAON INC',b'021542',000360206,b'333415',b'20102010',...,-0.095238,0.263158,-0.166667,0.050000,-0.047619,NaN,4.0,NaN,NaN,NaN
3,3,76868.0,199105.0,137.0,AAON,b'AAON INC',b'021542',000360206,b'333415',b'20102010',...,0.263158,-0.166667,0.050000,-0.047619,0.050000,NaN,4.0,NaN,NaN,NaN
4,4,76868.0,199106.0,138.0,AAON,b'AAON INC',b'021542',000360206,b'333415',b'20102010',...,-0.166667,0.050000,-0.047619,0.050000,-0.142857,NaN,4.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
402602,402602,14642.0,201908.0,476.0,LPG,b'DORIAN LPG LTD',b'317264',Y2106R110,b'483111',b'10102040',...,0.089552,-0.133825,-0.058394,0.103359,-0.011710,0.344193,NaN,0.054879,0.511885,0.165357
402603,402603,14642.0,201909.0,477.0,LPG,b'DORIAN LPG LTD',b'317264',Y2106R110,b'483111',b'10102040',...,-0.133825,-0.058394,0.103359,-0.011710,-0.050948,0.344193,NaN,0.007823,0.586331,0.283299
402604,402604,14642.0,201910.0,478.0,LPG,b'DORIAN LPG LTD',b'317264',Y2106R110,b'483111',b'10102040',...,-0.058394,0.103359,-0.011710,-0.050948,0.023720,1.385321,NaN,-0.002021,0.645208,0.372886
402605,402605,14642.0,201911.0,479.0,LPG,b'DORIAN LPG LTD',b'317264',Y2106R110,b'483111',b'10102040',...,0.103359,-0.011710,-0.050948,0.023720,0.332927,1.385321,NaN,-0.004380,0.554185,0.372508


In [5]:
# creating new df with all factors
df = a15[['permno', 'yyyymm', 'monthid', 'ticker', 'BM', 'beta_3y', 'IV_capm', 'RET', 'PRC', 'SHROUT', 'Sue', 'abr', 'Res', 'R6', 'R11']]
#df

In [6]:
a7_8_10 = pd.read_csv('factorsA7_A8_A10.csv')
a7_8_10

,permno,yyyymm,monthid,ticker,gvkey,cusip,naics,gsubind,IM,range_20,...,ret_f6,ret_f7,ret_f8,ret_f9,ret_f10,ret_f11,ret_f12,Rs,tes,Nei
0,10026,199001.0,121.0,JJSF,12825,466032109,311812,30202030,0.511680,0.407641,...,0.030303,-0.088235,-0.209677,-0.040816,0.489362,-0.100000,0.015873,-0.224550,0.068135,2
1,10026,199002.0,122.0,JJSF,12825,466032109,311812,30202030,0.524748,0.444449,...,-0.088235,-0.209677,-0.040816,0.489362,-0.100000,0.015873,0.375000,-0.225071,0.068135,2
2,10026,199003.0,123.0,JJSF,12825,466032109,311812,30202030,0.299829,0.520924,...,-0.209677,-0.040816,0.489362,-0.100000,0.015873,0.375000,0.056818,-1.219253,0.068135,0
3,10026,199004.0,124.0,JJSF,12825,466032109,311812,30202030,0.111335,0.303839,...,-0.040816,0.489362,-0.100000,0.015873,0.375000,0.056818,0.193548,-1.161494,-0.125056,0
4,10026,199005.0,125.0,JJSF,12825,466032109,311812,30202030,0.184506,0.378662,...,0.489362,-0.100000,0.015873,0.375000,0.056818,0.193548,-0.045045,-1.148628,-0.125056,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
297389,93429,201801.0,457.0,CBOE,184500,12503M108,523210,40203040,0.568230,0.089902,...,-0.066686,0.040976,-0.048016,0.176011,-0.043598,-0.090968,-0.046612,2.341509,0.407873,1
297390,93429,201803.0,459.0,CBOE,184500,12503M108,523210,40203040,0.562365,0.340422,...,-0.048016,0.176011,-0.043598,-0.090968,-0.046612,0.031629,-0.004900,2.157655,0.407873,1
297391,93429,201804.0,460.0,CBOE,184500,12503M108,523210,40203040,0.544807,0.193830,...,0.176011,-0.043598,-0.090968,-0.046612,0.031629,-0.004900,0.064648,2.130446,-10.138196,1
297392,93429,201805.0,461.0,CBOE,184500,12503M108,523210,40203040,0.455098,0.122062,...,-0.043598,-0.090968,-0.046612,0.031629,-0.004900,0.064648,0.071253,2.112371,-10.138196,1


In [7]:
a218_219 = pd.read_csv('factorsA218_A219.csv')
a218_219

,permno,yyyymm,monthid,ticker,gvkey,cusip,naics,gsubind,IM,range_20,...,ret_f6,ret_f7,ret_f8,ret_f9,ret_f10,ret_f11,ret_f12,year,Sg,Sr
0,10026,198603.0,75.0,JJSF,12825,466032109,311812,30202030,0.201498,0.146460,...,-0.166667,0.114286,0.051282,-0.048780,0.615385,0.031746,0.030769,1986.0,0.000000,0.0
1,10026,198604.0,76.0,JJSF,12825,466032109,311812,30202030,0.985464,0.158181,...,0.114286,0.051282,-0.048780,0.615385,0.031746,0.030769,-0.119403,1986.0,0.000000,0.0
2,10026,198605.0,77.0,JJSF,12825,466032109,311812,30202030,0.715998,0.158346,...,0.051282,-0.048780,0.615385,0.031746,0.030769,-0.119403,-0.042373,1986.0,0.000000,0.0
3,10026,198606.0,78.0,JJSF,12825,466032109,311812,30202030,0.672322,0.126541,...,-0.048780,0.615385,0.031746,0.030769,-0.119403,-0.042373,0.159292,1986.0,0.000000,0.0
4,10026,198607.0,79.0,JJSF,12825,466032109,311812,30202030,0.638727,0.127120,...,0.615385,0.031746,0.030769,-0.119403,-0.042373,0.159292,0.114504,1986.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
293288,93427,201908.0,476.0,FN,179583,G3323L100,3344,45203020,0.515433,0.124486,...,-0.125635,-0.010160,0.150110,0.018964,-0.023772,0.163569,-0.039240,2019.0,0.154826,0.0
293289,93427,201909.0,477.0,FN,179583,G3323L100,3344,45203020,0.427361,0.282884,...,-0.010160,0.150110,0.018964,-0.023772,0.163569,-0.039240,-0.096733,2019.0,0.154826,0.0
293290,93427,201910.0,478.0,FN,179583,G3323L100,3344,45203020,0.458960,0.219210,...,0.150110,0.018964,-0.023772,0.163569,-0.039240,-0.096733,-0.047755,2019.0,0.154826,0.0
293291,93427,201911.0,479.0,FN,179583,G3323L100,3344,45203020,0.452644,0.188740,...,0.018964,-0.023772,0.163569,-0.039240,-0.096733,-0.047755,0.138121,2019.0,0.154826,0.0


In [8]:
#dropping nans
factors = ['Sue', 'abr', 'Res', 'R6', 'R11']
df = df.dropna(subset=factors)
df

,permno,yyyymm,monthid,ticker,BM,beta_3y,IV_capm,RET,PRC,SHROUT,Sue,abr,Res,R6,R11
36,76868.0,199402.0,170.0,AAON,0.140243,1.060694,0.047747,0.202532,11.875000,5538.0,-0.479005,2.055649,0.007713,0.581189,1.557374
37,76868.0,199403.0,171.0,AAON,0.145603,0.840732,0.033314,0.010526,12.000000,5538.0,-0.479005,2.177212,0.016467,0.598315,1.752519
38,76868.0,199404.0,172.0,AAON,0.145603,0.931947,0.032722,0.385417,16.625000,5538.0,-0.175653,2.360957,-0.003306,0.481698,1.621717
39,76868.0,199405.0,173.0,AAON,0.112315,0.944933,0.031496,-0.030075,16.125000,5538.0,-0.175653,2.262519,0.001361,0.524482,1.298910
40,76868.0,199406.0,174.0,AAON,0.112315,0.941657,0.032464,-0.077519,14.875000,5543.0,-0.175653,2.250231,0.005496,0.959899,1.278076
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
168155,88865.0,201909.0,477.0,HAFC,0.857936,1.715368,0.014431,0.048576,18.780001,31174.0,0.898109,1.942709,0.000061,0.013624,-0.105476
168156,88865.0,201910.0,478.0,HAFC,0.857936,1.724765,0.014641,0.025027,19.250000,31174.0,0.898109,1.942709,0.000061,-0.194237,-0.214920
168157,88865.0,201910.0,478.0,HAFC,0.857936,1.724765,0.014641,0.025027,19.250000,31174.0,0.898109,1.942709,0.000061,-0.067238,-0.008914
168158,88865.0,201911.0,479.0,HAFC,0.989152,1.731579,0.013885,0.032208,19.629999,31114.0,0.898109,1.942709,0.002622,-0.157397,-0.064917


In [9]:
# nan for below factors have been dropped already
factors += ['Rs', 'tes', 'Nei']
factors += ['Sg', 'Sr']
df = pd.merge(df, a7_8_10[['permno', 'yyyymm', 'Rs', 'tes', 'Nei']], on=['permno', 'yyyymm'], how='inner')
df = pd.merge(df, a218_219[['permno', 'yyyymm', 'Sg', 'Sr']], on=['permno', 'yyyymm'], how='inner')
df.loc[:,'MKTCAP'] = df['PRC'] * 1000 * df['SHROUT']
df

,permno,yyyymm,monthid,ticker,BM,beta_3y,IV_capm,RET,PRC,SHROUT,...,abr,Res,R6,R11,Rs,tes,Nei,Sg,Sr,MKTCAP
0,76868.0,199504.0,184.0,AAON,0.160536,2.670385,0.028582,-0.254054,8.6250,6101.0,...,1.519529,0.009902,-0.089200,0.120533,2.390813,0.002184,1,0.752258,0.0,5.262112e+07
1,76868.0,199505.0,185.0,AAON,0.174546,2.415390,0.069457,-0.115942,7.6250,6101.0,...,1.472125,0.008405,0.009381,-0.247384,1.820668,0.002184,1,0.752258,0.0,4.652012e+07
2,76868.0,199506.0,186.0,AAON,0.174546,2.138157,0.037874,-0.065574,7.1250,6101.0,...,1.514372,0.007942,-0.372124,-0.471363,0.559510,0.002184,0,0.752258,0.0,4.346962e+07
3,76868.0,199507.0,187.0,AAON,0.174546,1.733739,0.021850,0.052632,7.5000,6101.0,...,1.491174,0.007555,-0.461979,-0.509785,0.558384,-0.000497,0,0.752258,0.0,4.575750e+07
4,76868.0,199508.0,188.0,AAON,0.302441,1.070538,0.017690,-0.041667,7.1875,6101.0,...,1.438134,0.007555,-0.447196,-0.558552,-0.618569,-0.000497,0,0.752258,0.0,4.385094e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81793,91416.0,201908.0,476.0,HBI,0.160610,1.374485,0.019933,-0.141703,13.6600,361543.0,...,-0.614137,0.000528,0.408630,-0.091386,1.939754,-0.109514,2,0.051387,5.2,4.938677e+09
81794,91416.0,201909.0,477.0,HBI,0.182319,1.440109,0.020695,0.121523,15.3200,361543.0,...,-0.726242,-0.002966,0.146680,0.048293,0.652162,-0.109514,1,0.051387,5.2,5.538839e+09
81795,91416.0,201910.0,478.0,HBI,0.182319,1.453799,0.026412,-0.007180,15.2100,361697.0,...,-0.503221,-0.004251,-0.245189,-0.144151,0.734443,0.049933,1,0.051387,5.2,5.501411e+09
81796,91416.0,201911.0,479.0,HBI,0.222112,1.500993,0.018886,0.000657,15.0700,361697.0,...,-0.202738,-0.002336,-0.085474,0.046281,4.209494,0.049933,1,0.051387,5.2,5.450774e+09


In [10]:
# filter by price and market cap
df = df.groupby(['monthid', 'ticker']).filter(condition_first_element)

In [11]:
df

,permno,yyyymm,monthid,ticker,BM,beta_3y,IV_capm,RET,PRC,SHROUT,...,abr,Res,R6,R11,Rs,tes,Nei,Sg,Sr,MKTCAP
48,76868.0,200004.0,244.0,AAON,0.373645,1.246686,0.020231,0.195730,21.000,5889.0,...,-0.306811,0.004365,0.388867,0.503860,3.063997,0.003269,2,0.199043,5.133333,1.236690e+08
49,76868.0,200005.0,245.0,AAON,0.305543,1.052334,0.030313,0.160714,24.375,5889.0,...,-0.301422,0.009177,0.281329,0.476346,5.933247,0.003269,4,0.199043,5.133333,1.435444e+08
50,76868.0,200006.0,246.0,AAON,0.305543,0.943883,0.028784,0.005128,24.500,5850.0,...,-0.372652,0.009177,0.504836,0.763473,5.822836,0.003269,4,0.199043,5.133333,1.433250e+08
51,76868.0,200007.0,247.0,AAON,0.305543,0.884476,0.038987,-0.025510,23.875,5850.0,...,-0.585151,0.008832,0.703646,0.835430,5.721207,0.003962,4,0.199043,5.133333,1.396688e+08
52,76868.0,200008.0,248.0,AAON,0.229883,1.021147,0.020963,0.026178,24.500,5850.0,...,-0.561189,-0.001813,0.570160,0.639471,6.646016,0.003962,4,0.199043,5.133333,1.433250e+08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81793,91416.0,201908.0,476.0,HBI,0.160610,1.374485,0.019933,-0.141703,13.660,361543.0,...,-0.614137,0.000528,0.408630,-0.091386,1.939754,-0.109514,2,0.051387,5.200000,4.938677e+09
81794,91416.0,201909.0,477.0,HBI,0.182319,1.440109,0.020695,0.121523,15.320,361543.0,...,-0.726242,-0.002966,0.146680,0.048293,0.652162,-0.109514,1,0.051387,5.200000,5.538839e+09
81795,91416.0,201910.0,478.0,HBI,0.182319,1.453799,0.026412,-0.007180,15.210,361697.0,...,-0.503221,-0.004251,-0.245189,-0.144151,0.734443,0.049933,1,0.051387,5.200000,5.501411e+09
81796,91416.0,201911.0,479.0,HBI,0.222112,1.500993,0.018886,0.000657,15.070,361697.0,...,-0.202738,-0.002336,-0.085474,0.046281,4.209494,0.049933,1,0.051387,5.200000,5.450774e+09


**5. Uniformization and Data Filtering**

In [12]:
#getting z scores
def getZ(df, factors):
    for f in factors:
        z_scores = (df[f] - df[f].mean()) / df[f].std()
        # Add z-scored column to the DataFrame
        df.loc[:,f + '_z_score'] = z_scores
    return df

In [13]:
df = getZ(df, factors)
#df

**6. Fama-MacBeth and ML**

In [14]:
# lnSize
df.loc[:,'lnSize'] = np.log(df['MKTCAP'])

# BM - bk2mkt
# beta_3y - beta
# IV_capm - ivol

# rename
df.rename(columns = {
    'BM': 'bk2mkt',
    'beta_3y': 'beta',
    'IV_capm': 'ivol',
}, inplace=True)
#df

In [15]:
variables = ['beta', 'lnSize', 'bk2mkt', 'ivol']
def winsorize(df, variables):
    for var in variables:
        mean_value = np.mean(df[var])
        std_dev = np.std(df[var])

        # Calculate lower and upper bounds
        lower_bound = mean_value - 3 * std_dev
        upper_bound = mean_value + 3 * std_dev
        
        df.loc[:,var+'_winsorized'] = np.clip(df[var], lower_bound, upper_bound)
    return df

df = winsorize(df, variables)
df

,permno,yyyymm,monthid,ticker,bk2mkt,beta,ivol,RET,PRC,SHROUT,...,Rs_z_score,tes_z_score,Nei_z_score,Sg_z_score,Sr_z_score,lnSize,beta_winsorized,lnSize_winsorized,bk2mkt_winsorized,ivol_winsorized
48,76868.0,200004.0,244.0,AAON,0.373645,1.246686,0.020231,0.195730,21.000,5889.0,...,0.865799,-0.004614,0.218885,NaN,0.517946,18.633119,1.246686,18.633119,0.373645,0.020231
49,76868.0,200005.0,245.0,AAON,0.305543,1.052334,0.030313,0.160714,24.375,5889.0,...,2.268507,-0.004614,1.262162,NaN,0.517946,18.782155,1.052334,18.782155,0.305543,0.030313
50,76868.0,200006.0,246.0,AAON,0.305543,0.943883,0.028784,0.005128,24.500,5850.0,...,2.214530,-0.004614,1.262162,NaN,0.517946,18.780625,0.943883,18.780625,0.305543,0.028784
51,76868.0,200007.0,247.0,AAON,0.305543,0.884476,0.038987,-0.025510,23.875,5850.0,...,2.164846,-0.004613,1.262162,NaN,0.517946,18.754784,0.884476,18.754784,0.305543,0.038987
52,76868.0,200008.0,248.0,AAON,0.229883,1.021147,0.020963,0.026178,24.500,5850.0,...,2.616963,-0.004613,1.262162,NaN,0.517946,18.780625,1.021147,18.780625,0.229883,0.020963
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81793,91416.0,201908.0,476.0,HBI,0.160610,1.374485,0.019933,-0.141703,13.660,361543.0,...,0.316183,-0.004855,0.218885,NaN,0.547490,22.320363,1.374485,22.320363,0.160610,0.019933
81794,91416.0,201909.0,477.0,HBI,0.182319,1.440109,0.020695,0.121523,15.320,361543.0,...,-0.313291,-0.004855,-0.302753,NaN,0.547490,22.435051,1.440109,22.435051,0.182319,0.020695
81795,91416.0,201910.0,478.0,HBI,0.182319,1.453799,0.026412,-0.007180,15.210,361697.0,...,-0.273065,-0.004515,-0.302753,NaN,0.547490,22.428271,1.453799,22.428271,0.182319,0.026412
81796,91416.0,201911.0,479.0,HBI,0.222112,1.500993,0.018886,0.000657,15.070,361697.0,...,1.425805,-0.004515,-0.302753,NaN,0.547490,22.419023,1.500993,22.419023,0.222112,0.018886


In [16]:
# Fama Macbeth
def fmRegression(df, factor):
    fm = df[['permno', 'yyyymm', factor, 'ivol_winsorized', 'beta_winsorized', 'lnSize_winsorized', 'bk2mkt_winsorized']]
    fm = fm.dropna(inplace=False)
    # Step 1: regress each of returns of each asset against the factors to determine each asset's beta exposures.
    groups = fm.groupby(['permno'])
    assetBetas = {}
    for name, group in groups:
        #print(name)
        X = group[['ivol_winsorized', 'beta_winsorized', 'lnSize_winsorized', 'bk2mkt_winsorized']]
        X = sm.add_constant(X)
        y = group[factor]
        model = sm.OLS(y, X)
        assetBetas[name] = model.fit().params
    
    # Settings up second step using beta exposures
    
    betaDf = pd.DataFrame(assetBetas)
    betaDf = betaDf.transpose()
    betaDf.reset_index(inplace=True)
    newColumns = ['permno', 'const', 'ivol_beta', 'beta_beta', 'lnSize_beta', 'bk2mkt_beta']
    betaDf.columns = newColumns
    
    fm = pd.merge(fm, betaDf, on='permno')
    fm = fm.dropna(inplace=False)
    # Step 2: regress all asset returns for each of T time periods against the previously estimated betas to determine the risk premium for each factor.
    groups = fm.groupby(['yyyymm'])
    
    # RESULTS IS PREMIUM HERE
    results = []
    for name, group in groups:
        X = group[['ivol_beta', 'beta_beta', 'lnSize_beta', 'bk2mkt_beta']]
        X = sm.add_constant(X)
        y = group[factor]
        model = sm.OLS(y, X).fit()
        results.append(model.params['ivol_beta'])
    return results

**7. Average Factor Premium and T-statistics**

In [17]:
for f in factors:
    res = fmRegression(df, f)
    mean = np.mean(res)
    t, p = stats.ttest_1samp(res, 0)
    print(f, "mean:", mean, "t-stat:", t, 'p-value: ', p)

Sue mean: 0.11573395086321393 t-stat: 18.583127679649007 p-value:  3.7171801655240556e-56
abr mean: 0.14120313820078806 t-stat: 30.835959522579103 p-value:  3.3228618792498388e-108
Res mean: 0.10181156480623152 t-stat: 1.6888395361811257 p-value:  0.09202179152605634
R6 mean: -0.025736708066907456 t-stat: -4.444425015026853 p-value:  1.1398488427222415e-05
R11 mean: -0.03187573219864283 t-stat: -7.409872572122451 p-value:  7.460850465657721e-13
Rs mean: -0.05130880801389706 t-stat: -14.4160207252371 p-value:  2.640643464656267e-38
tes mean: -0.006170700153644048 t-stat: -0.24106726179523846 p-value:  0.8096252810769624
Nei mean: 0.0036855947515208043 t-stat: 1.4884250734283815 p-value:  0.13741896577086407
Sg mean: -0.07271366854247809 t-stat: -5.019942769205349 p-value:  7.766008935902998e-07
Sr mean: 0.01905677268796669 t-stat: 5.310280408650402 p-value:  1.8110793499167013e-07
